In [1]:
import plotly.dashboard_objs as dashboard

import IPython.display
from IPython.display import Image

import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
import ipywidgets as widgets
import numpy as np
from scipy import special
from ipywidgets import interact
import time
import pandas as pd
import matplotlib.pyplot as plt
import re
plt.style.use('ggplot')
%matplotlib inline

devices_data = pd.read_csv('C:\\Users\\ruben\\Desktop\\Investigation\\updated_dataset_07_3_2018\\devices.csv', index_col='id', parse_dates=['created_at', 'updated_at'])
samples_data = pd.read_csv('C:\\Users\\ruben\\Desktop\\Investigation\\updated_dataset_07_3_2018\\samples.csv', usecols=['device_id', 'timestamp', 'battery_state', 'battery_level'], parse_dates=['timestamp'])

tls.set_credentials_file(username='hawnkys', api_key='SzhJtr7f8NebSmaJHCJW')

my_dboard = dashboard.Dashboard()

In [2]:
all_versions_percentage = devices_data.os_version.value_counts(normalize=True) #get all version percentage

layout = go.Layout(
    title='Android Version Percentages',
    yaxis=dict(
        title='Percentage'
    ),
    xaxis=dict(
        title='Android Version'
    )
)

data = go.Bar(
        x = all_versions_percentage.index,
        y = all_versions_percentage.values
)

data = [data]
url_1 = py.plot(data, filename='android_version_percenatges', auto_open=False)
py.iplot(data, filename='android_version_percenatges')

In [3]:
def getVersionCounter(all_versions_counts):
    version_counts = dict()
    
    for index_val, series_val in all_versions_counts.iteritems():
        if index_val[0] in version_counts:
            version_counts[index_val[0]] += series_val
        else:
            version_counts[index_val[0]] = series_val
    
    return version_counts

version_percentage = getVersionCounter(all_versions_percentage) #join all versions

keys, values = zip(*version_percentage.items()) #gets 2 lists one with keys another with values od dictionary
values = [x * 100 for x in values] #convert to percentage (int)

layout2 = go.Layout(
    title='Devices Android Version',
    yaxis=dict(
        title='Percentage'
    ),
    xaxis=dict(
        title='Version'
    )
)

data2 = go.Bar(
            x = keys,
            y = values
    )



data = [data2]
url_2 = py.plot(data, filename='android_version', auto_open=False)
py.iplot(data, filename='android_version')

In [4]:
from plotly.widgets import GraphWidget
from ipywidgets.embed import embed_minimal_html



# computes a list of [discharging rate per hour (percentage), time to discharge 1% (seconds)] of given discharging sample, ex: [0.41, 356]
def computeDischargeKPI(valid_sample):
    previous_state_index = 0
    battery_per_hour = []
    discharge_per_unit = 0
    
    for index, row in valid_sample.iterrows():
        if index != 0:
            discharge_per_unit += (row['timestamp'] - valid_sample['timestamp'].iloc[index - 1]).seconds
        
        if (row['timestamp'] - valid_sample['timestamp'].iloc[previous_state_index]).seconds >= 3600:
            battery_per_hour.append(abs(row['battery_level'] - valid_sample['battery_level'].iloc[previous_state_index]))
            previous_state_index = index

    if not battery_per_hour:
        return []
    
    return [sum(battery_per_hour) / float(len(battery_per_hour)), discharge_per_unit/len(valid_sample.index)]


#time to charge 1% (seconds) of given charging sample
def computeChargeKPI(valid_sample):
    previous_state_index = 0
    charge_per_unit = 0
    
    for index, row in valid_sample.iterrows():
        if index != 0:
            charge_per_unit += (row['timestamp'] - valid_sample['timestamp'].iloc[index - 1]).seconds
    return charge_per_unit/len(valid_sample.index)


#check if is charge/discharge >= given percentage and if they the step checks (step=0, continuous sample)
def checkSampleIsValid(subset, percentage, step):
    return abs(subset['battery_level'].iloc[len(subset) - 1] - subset['battery_level'].iloc[0]) >= percentage and (len(subset) - 1 == abs(subset['battery_level'].iloc[len(subset) - 1] - subset['battery_level'].iloc[0]) * 100)


def computeDeviceKPI(devID, percentage, step):
    device_samples = samples_data.loc[samples_data.device_id == devID].drop_duplicates(subset=['timestamp']).sort_values(by='timestamp').reset_index(drop=True)
    
    if device_samples.empty:
        return [-1, -1, -1, -1, -1]
    
    charge_to_full = 0
    charge_to_not_full = 0

    results_descharging = []
    results_charging = []
    
    previous_state_index = 0
    for index, row in device_samples.iterrows():
        #check diference on battery state to divide the samples into subsets
        if row['battery_state'] != device_samples['battery_state'].iloc[previous_state_index]:
            #creates substet only with charging samples ending in full state 
            if row['battery_state'] == 'Full':
                subset = device_samples.loc[previous_state_index:index].drop_duplicates(subset=['battery_level'], keep='last').reset_index(drop=True)
                
                #check if samples are valid and discard not charging samples
                if 'Not charging' not in subset['battery_state'].unique() and checkSampleIsValid(subset, percentage, step):
                    charge_to_full += 1
                    results_charging.append(computeChargeKPI(subset))
        
                previous_state_index = index + 1

            #create all others subsets (discharging, not charging samples, only charging, only full)
            else:
                subset = device_samples[previous_state_index:index].drop_duplicates(subset=['battery_level']).reset_index(drop=True)
                
                if subset['battery_state'].iloc[0] == 'Charging':
                    charge_to_not_full += 1
                    
                #check if samples are valid and discard not charging samples
                if 'Not charging' not in subset['battery_state'].unique() and checkSampleIsValid(subset, percentage, step):
                     if subset['battery_state'].iloc[0] == 'Discharging':
                        res = computeDischargeKPI(subset)
                        if len(res) != 0:
                            results_descharging.append(res)
                        else:
                            results_charging.append(computeChargeKPI(subset))
                        
                previous_state_index = index
                
        #check if is the last sample to compute the last substet
        if index == len(device_samples) - 1:
            subset = device_samples.loc[previous_state_index:index].drop_duplicates(subset=['battery_level'], keep='last').reset_index(drop=True)
                
            #check if samples are valid and discard not charging samples
            if not subset.empty and 'Not charging' not in subset['battery_state'].unique() and checkSampleIsValid(subset, percentage, step):
                if subset['battery_state'].iloc[0] == 'Discharging':
                    res = computeDischargeKPI(subset)
                    if len(res) != 0:
                        results_descharging.append(res)
                    else:
                        results_charging.append(computeChargeKPI(subset))

    if len(results_charging) != 0 and len(results_descharging) != 0:
        return [time.strftime("%M:%S", time.gmtime(sum(results_charging) / float(len(results_charging)))), time.strftime("%M:%S", time.gmtime(sum([row[1] for row in results_descharging])/len(results_descharging))), sum([row[0] for row in results_descharging])/len(results_descharging) , charge_to_full, charge_to_not_full]  # adding a row
    elif len(results_charging) != 0:
        return [time.strftime("%M:%S", time.gmtime(sum(results_charging) / float(len(results_charging)))), -1, -1, charge_to_full, charge_to_not_full]  # adding a row
    elif len(results_descharging) != 0:
        return [-1, time.strftime("%M:%S", time.gmtime(sum([row[1] for row in results_descharging])/len(results_descharging))), sum([row[0] for row in results_descharging])/len(results_descharging) , charge_to_full, charge_to_not_full]  # adding a row
    else:
        return [-1, -1, -1, -1, -1]  # adding a row

#-----------------------------------------------------------------------------------------------


kpis_res = computeDeviceKPI(1, 0.5, 0)

kpis_res[2] = round(kpis_res[2]*100)


@interact(drop_options = widgets.Dropdown(options=[str(i) for i in list(devices_data.index)], value='1', description='Device ID:', disabled=False,))
def update_plot(drop_options):
    data = []
    
    for s in drop_options:
        trace = go.Table(
            header=dict(values=['Device KPI', 'Value'],
                        line = dict(color='#7D7F80'),
                        fill = dict(color='#a1c3d1'),
                        align = ['left'] * 5),
            cells=dict(values=[['Average time to charge 1%', 'Average time to discharge 1%', 'Average discharge rate per hour', 'Reached full state', 'Charging does not reached full state'],
                       computeDeviceKPI(int(s), 0.5, 0)],
                       line = dict(color='#7D7F80'),
                       fill = dict(color='#EDFAFF'),
                       align = ['left'] * 5))

    layout = dict(width=650, height=350)
    data = [trace]

    url_3 = py.plot(data, filename='table_kpi', auto_open=False)
    py.plot(data, filename='table_kpi')



<IPython.core.display.Javascript object>

A Jupyter Widget

In [8]:
def fileId_from_url(url):
    """Return fileId from a url."""
    raw_fileId = re.findall("~[A-z]+/[0-9]+", url)[0][1: ]
    return raw_fileId.replace('/', ':')

fileId_1 = fileId_from_url(url_1)
fileId_2 = fileId_from_url(url_2)
fileId_3 = fileId_from_url(url_3)

box_a = {
    'type': 'box',
    'boxType': 'plot',
    'fileId': fileId_1,
    'title': 'Android Version Percentages'
}

box_b = {
    'type': 'box',
    'boxType': 'plot',
    'fileId': fileId_2,
    'title': 'Android Version'
}

box_c = {
    'type': 'box',
    'boxType': 'table',
    'fileId': fileId_2,
    'title': 'Android Version'
}

my_dboard.insert(box_c, 'above', 1)
my_dboard.insert(box_a, 'above', 1)
my_dboard.insert(box_b, 'left', 1, fill_percent=50)

my_dboard.get_preview()

In [ ]:
my_dboard['settings']['title'] = 'GreenHub Dashboard'

In [11]:
py.dashboard_ops.upload(my_dboard, 'My First Dashboard with Python')

PlotlyRequestError: {u'direction': u'vertical', u'sizeUnit': u'px', u'second': {u'boxType': u'empty', u'type': u'box'}, u'size': 1850, u'type': u'split', u'first': {u'direction': u'vertical', u'sizeUnit': u'%', u'second': {u'boxType': u'table', u'title': u'Android Version', u'type': u'box', u'fileId': u'hawnkys:19'}, u'size': 50, u'type': u'split', u'first': {u'direction': u'horizontal', u'sizeUnit': u'%', u'second': {u'boxType': u'plot', u'title': u'Android Version Percentages', u'type': u'box', u'fileId': u'hawnkys:17'}, u'size': 50, u'type': u'split', u'first': {u'boxType': u'plot', u'title': u'Android Version', u'type': u'box', u'fileId': u'hawnkys:19'}}}} is not valid under any of the given schemas

Failed validating 'oneOf' in schema['properties']['layout']:
    {'oneOf': [{'type': 'null'}, {'$ref': '#/definitions/container'}]}

On instance['layout']:
    {u'direction': u'vertical',
     u'first': {u'direction': u'vertical',
                u'first': {u'direction': u'horizontal',